In [137]:
import requests, json, sys, folium
import pandas as pd
from postcodes_api import PosscodeApi

In [143]:
sch_stat = pd.read_excel('Datasets/scottish_schools_stats.xlsx')
sch_info = pd.read_excel('Datasets/scottish_schools_contact.xlsx', sheet_name='Open Schools')
dep = pd.read_excel('Datasets/postcode_deprivation.xlsx')

In [145]:
dep_rates = {}

for p,d in dep.values:
    dep_rates[p] = d

In [31]:
sch_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Local Authority  2461 non-null   object
 1   SeedCode         2461 non-null   int64 
 2   School Name      2461 non-null   object
 3   School Type      2461 non-null   object
 4   Total pupils     2461 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 96.3+ KB


In [32]:
sch_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Seed Code                                2458 non-null   int64 
 1   LA Name                                  2458 non-null   object
 2   Centre Type                              2458 non-null   object
 3   School Name                              2458 non-null   object
 4   Address Line1                            2458 non-null   object
 5   Address Line2                            2457 non-null   object
 6   Address Line3                            2458 non-null   object
 7   Post Code                                2458 non-null   object
 8   Unique Property Reference Number (UPRN)  2458 non-null   int64 
 9   Email                                    2458 non-null   object
 10  Phone Number                             2458 non-null   obj

In [34]:
sch_info = sch_info.rename(columns={'Seed Code' : 'SeedCode'})

In [37]:
sch_df = sch_info.merge(sch_stat, on='SeedCode', how='right')

In [43]:
sch_df.isna().sum()
sch_df.dropna(inplace=True)

In [64]:
sch_jsn = json.loads(sch_df.to_json(orient='records'))

In [67]:
sch_jsn[0].keys()

dict_keys(['SeedCode', 'LA Name', 'Centre Type', 'School Name_x', 'Address Line1', 'Address Line2', 'Address Line3', 'Post Code', 'Unique Property Reference Number (UPRN)', 'Email', 'Phone Number', 'Website Address', 'Pre-school Department', 'Primary Department', 'Secondary Department', 'Special Department', 'Gaelic Unit', 'Integrated Special Unit', 'Denomination', 'Local Authority', 'School Name_y', 'School Type', 'Total pupils'])

In [141]:
st_lat = 55.941457
st_lon = -3.205744


m = folium.Map(location=[st_lat, st_lon], zoom_start=12)

for sch in sch_jsn:

    pos = sch['Post Code']
    if pos[:2].upper() == 'EH':
        try:
            loc = PosscodeApi().get_pos_info(pos)['loc']
        except TypeError:
            pass
        city = sch['LA Name']
        
        folium.Marker(
            location=loc,
            popup=city,
            icon=folium.Icon(color='blue')
        ).add_to(m)


# Display the map (optional, this works in Jupyter Notebook)
m

The postcode EH33 2LX is terminated.
Non-valid post code: EH7 4TN


In [142]:
sch_jsn[0]

{'SeedCode': 1002929,
 'LA Name': 'Aberdeen City',
 'Centre Type': 'Local Authority',
 'School Name_x': 'Brimmond School',
 'Address Line1': 'Netherhills Place',
 'Address Line2': 'Bucksburn',
 'Address Line3': 'ABERDEEN',
 'Post Code': 'AB21 9DF',
 'Unique Property Reference Number (UPRN)': 9051145398.0,
 'Email': 'brimmondschool@aberdeencity.gov.uk',
 'Phone Number': '01224 719930',
 'Website Address': 'https://www.brimmond.aberdeen.sch.uk',
 'Pre-school Department': 'Yes',
 'Primary Department': 'Yes',
 'Secondary Department': 'No',
 'Special Department': 'No',
 'Gaelic Unit': 'No',
 'Integrated Special Unit': 'Yes',
 'Denomination': 'Non-denominational',
 'Local Authority': 'Aberdeen City',
 'School Name_y': 'Brimmond School',
 'School Type': 'Primary',
 'Total pupils': 425}

In [152]:
sch_jsn[0]

{'SeedCode': 1002929,
 'LA Name': 'Aberdeen City',
 'Centre Type': 'Local Authority',
 'School Name_x': 'Brimmond School',
 'Address Line1': 'Netherhills Place',
 'Address Line2': 'Bucksburn',
 'Address Line3': 'ABERDEEN',
 'Post Code': 'AB21 9DF',
 'Unique Property Reference Number (UPRN)': 9051145398.0,
 'Email': 'brimmondschool@aberdeencity.gov.uk',
 'Phone Number': '01224 719930',
 'Website Address': 'https://www.brimmond.aberdeen.sch.uk',
 'Pre-school Department': 'Yes',
 'Primary Department': 'Yes',
 'Secondary Department': 'No',
 'Special Department': 'No',
 'Gaelic Unit': 'No',
 'Integrated Special Unit': 'Yes',
 'Denomination': 'Non-denominational',
 'Local Authority': 'Aberdeen City',
 'School Name_y': 'Brimmond School',
 'School Type': 'Primary',
 'Total pupils': 425}

In [171]:
import folium
from folium.plugins import HeatMap

# Create a map centered at a specific location
m = folium.Map(location=[55.941457, -3.205744], zoom_start=12)  # London, UK

# Create a custom color scale from light to dark blue
colors = {
    1: '#08306b',  # Light blue
    2: '#08519c',
    3: '#3182bd',
    4: '#63b7f4',
    5: '#a6e1fa'   # Dark blue (most deprived)
}

# Create circles and digits for each data point
for sch in sch_jsn:

    if sch['LA Name'] == 'Edinburgh City':

        pos = sch['Post Code']

        try:
            loc = PosscodeApi().get_pos_info(pos)['loc']
        except TypeError:
            pass

    else:
        continue

    try:
        mag = dep_rates[pos] # magnitute
    except:
        mag = 3 # average
    
    folium.CircleMarker(
        location=loc,
        radius=25//mag,
        color=colors[mag],
        fill=True,
        fill_opacity=0.8,
    ).add_to(m)

    popup_html = f"""
    <h3>{sch['School Name_x']}</h3>
    <p><strong>Local Authority:</strong> {sch["LA Name"]}</p>
    <p><strong>Pupils:</strong> {sch["Total pupils"]}</p>
    <p><strong>Deprivation:</strong> {mag}</p>"""

    folium.Marker(
        location=loc,
        popup=folium.Popup(popup_html, max_width=150),
        icon=folium.DivIcon(html=f'<div style="width: 0px; height: 0px;"></div>'),
    ).add_to(m)

# Create a HeatMap layer using some sample data points
# heat_data = [[51.5074, -0.1278, 0.5], [51.5099, -0.1337, 0.2], [51.5113, -0.1219, 0.3]]
# HeatMap(heat_data).add_to(m)

# Create a custom HTML legend
legend_html = """
<div style="position: fixed; top: 10px; right: 10px; background-color: white; padding: 10px; border: 2px solid black; z-index: 1000;">
    <p><strong>Legend</strong></p>
    <p><span style="color: black;"><span style="background-color: #08306b; width: 20px; height: 20px; display: inline-block;"></span> 1 - Least Deprived</span></p>
    <p><span style="color: black;"><span style="background-color: #08519c; width: 20px; height: 20px; display: inline-block;"></span> 2</span></p>
    <p><span style="color: black;"><span style="background-color: #3182bd; width: 20px; height: 20px; display: inline-block;"></span> 3</span></p>
    <p><span style="color: black;"><span style="background-color: #63b7f4; width: 20px; height: 20px; display: inline-block;"></span> 4</span></p>
    <p><span style="color: black;"><span style="background-color: #a6e1fa; width: 20px; height: 20px; display: inline-block;"></span> 5 - Most Deprived</span></p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


# Display the map (optional, this works in Jupyter Notebook)
m